In [1]:
import sqlite3
import json

In [56]:
# load in the sqlite file into memory

db_file= "scan_data.sqlite"

# basic setup
con = sqlite3.connect(db_file)
con.row_factory = sqlite3.Row
cur = con.cursor()

# join on the ID, as the base geometry BLOB in the ScanData table is raw binary
# we could convert that to geomtry via a QGIS package for JSON, but joining works just as well
db_rows = con.execute(
    "SELECT ScanData.pkuid, ScanData.data, ScanData.level, \
    idx_ScanData_geometry.xmin, idx_ScanData_geometry.xmax, idx_ScanData_geometry.ymin, idx_ScanData_geometry.ymax \
    FROM ScanData \
    INNER JOIN idx_ScanData_geometry \
    ON ScanData.pkuid=idx_ScanData_geometry.pkid"
)
# turn it into our favourite thing ever, a dictionary :)
db_dict = [dict(row) for row in db_rows]
print(f"Obtained {len(db_dict)} rows from database.")

# ensure the connection is closed
con.close()

Obtained 49 rows from database.


In [86]:
# load in the AP locational data for referencing

# we dont care about the preamble, so just get the features
with open("heuristic_nodes.json", 'r') as f:
    node_data = json.load(f)['features']
    
# will hold the AP locational data for referencing later
node_dict = {}

# set up dictionary with the MAC as the key, and the level and coordinates as values
for entry in node_data:
    data = entry['properties']
    
    # only take Points with a MAC value, i.e. only take APs
    if not data['mac_addres'] is None:
        
        # skim off the last char of the key. still unique. this makes later processing O(1) instead of O(n)
        node_dict[data['mac_addres'][:-1]] = {
            "level": int(data['level']),
            "coordinates": entry['geometry']['coordinates']
        }

# sort based on level
node_dict = dict(sorted(node_dict.items(), key=lambda x:x[1]['level']))        

for mac, data in node_dict.items():
    print(f"MAC {mac} on Level {data['level']} := {data['coordinates']}")

MAC 5c:a6:2d:ce:b1:a on Level 0 := [-1.55432572370256, 53.8087411724361]
MAC 5c:a6:2d:cf:c4:a on Level 0 := [-1.554222968311731, 53.80872998125492]
MAC 5c:a6:2d:ce:b8:2 on Level 0 := [-1.554165486335672, 53.808742698506265]
MAC ac:3a:67:08:26:2 on Level 0 := [-1.554090708897792, 53.808742698506265]
MAC ac:3a:67:08:06:2 on Level 0 := [-1.554323688942345, 53.80881594987398]
MAC 5c:a6:2d:86:69:6 on Level 0 := [-1.554220933551516, 53.80880984559334]
MAC 5c:a6:2d:cf:e0:6 on Level 0 := [-1.554121738991062, 53.808806793453016]
MAC ac:3a:67:07:fc:2 on Level 0 := [-1.554319619421916, 53.80895075273819]
MAC 5c:a6:2d:cf:ab:a on Level 0 := [-1.554218898791302, 53.80893803548685]
MAC 5c:a6:2d:af:8d:e on Level 0 := [-1.554162942885404, 53.8089660134398]
MAC ac:3a:67:08:32:4 on Level 0 := [-1.554086639377363, 53.80895533094867]
MAC 5c:a6:2d:cf:c3:c on Level 0 := [-1.554322671562238, 53.80905401681907]
MAC 5c:a6:2d:ef:4c:4 on Level 0 := [-1.554182273107442, 53.80905961240966]
MAC 5c:a6:2d:af:92:2 on L

In [91]:
def compare_networks(data):
    
    for scan in data:
        for network in scan:
            
            mac = network['BSSID']
            print(mac)
    
    return data

In [92]:
# turn sqlite file into dictionary, referencing scan data against AP dict to add locations

# the end dict that will hold all data for trilateration
ap_dict = {}

for row in db_dict:
    
    # point is a circle defined by bounding box - get center
    x = (row['xmin'] + row['xmax']) / 2
    y = (row['ymin'] + row['ymax']) / 2
    
    # main key for this dict will be by floor
    level = row['level']
    if not level in ap_dict:
        ap_dict[level] = {}
    
    # augment scan data with positional data for each scanned AP
    ap_dict[level][(x, y)] = compare_networks(json.loads(row['data']))
    
# sort based on floor, and display end message
ap_dict = dict(sorted(ap_dict.items()))
print(f"Dict created, with {len(ap_dict)} keys with the following value lengths: \
{[len(val) for val in ap_dict.values()]}")

#for level, level_data in ap_dict.items():
#    print(f"Level {level}")
#    for geometry, scan in level_data.items():
#        print(f"({geometry}) := '<scan data omitted for readability>")

5c:a6:2d:86:6c:a2
5c:a6:2d:af:94:42
5c:a6:2d:af:93:41
5c:a6:2d:af:93:42
ac:3a:67:07:f9:21
ac:3a:67:07:f9:22
5c:a6:2d:af:93:4e
5c:a6:2d:af:94:4d
5c:a6:2d:af:94:4e
5c:a6:2d:cf:b4:ae
5c:a6:2d:cf:c1:82
ac:3a:67:08:25:ae
ac:3a:67:08:25:ad
ac:3a:67:08:25:a2
5c:a6:2d:cf:b5:0e
7a:dc:24:13:09:d7
5c:a6:2d:86:6c:ae
5c:a6:2d:86:6c:ad
5c:a6:2d:af:94:41
5c:a6:2d:af:94:42
5c:a6:2d:af:93:42
ac:3a:67:07:f9:22
5c:a6:2d:cf:b4:a2
ac:3a:67:08:29:02
ac:3a:67:08:29:0d
ac:3a:67:08:29:0e
5c:a6:2d:86:6c:a2
5c:a6:2d:af:94:42
5c:a6:2d:af:93:42
5c:a6:2d:cf:b4:a2
ac:3a:67:07:f9:22
ac:3a:67:08:29:02
5c:a6:2d:5d:2e:21
5c:a6:2d:86:6c:a2
5c:a6:2d:86:6c:ae
5c:a6:2d:86:6c:ad
5c:a6:2d:af:94:41
5c:a6:2d:af:94:42
5c:a6:2d:cf:b4:a2
ac:3a:67:07:f9:22
5c:a6:2d:af:93:4d
5c:a6:2d:af:93:4e
5c:a6:2d:af:94:4d
5c:a6:2d:af:94:4e
ac:3a:67:07:f9:2e
ac:3a:67:08:29:02
5c:a6:2d:cf:b4:ad
ac:3a:67:08:25:ad
5c:a6:2d:cf:c1:82
ac:3a:67:08:25:ae
ac:3a:67:08:29:ad
ac:3a:67:08:29:0d
ac:3a:67:08:29:0e
5c:a6:2d:86:6c:a2
5c:a6:2d:af:94:42
5c:a6:2d:a

5c:a6:2d:af:91:c1
5c:a6:2d:86:6c:a1
5c:a6:2d:5d:2e:21
ac:3a:67:08:28:4e
5c:a6:2d:70:d0:ae
5c:a6:2d:ce:b4:c2
ac:3a:67:08:28:4d
5c:a6:2d:70:d0:ad
5c:a6:2d:ce:bf:02
ac:3a:67:08:29:ad
ac:3a:67:08:29:02
ac:3a:67:08:04:02
ac:3a:67:08:04:01
ac:3a:67:08:25:a2
5c:a6:2d:cf:b4:ad
5c:a6:2d:cf:b4:ae
ac:3a:67:08:25:ae
ac:3a:67:08:25:ad
5c:a6:2d:cf:b4:a1
5c:a6:2d:cf:b4:a2
ac:3a:67:08:29:0d
ac:3a:67:08:29:0e
5c:a6:2d:af:94:42
5c:a6:2d:cf:e7:42
5c:a6:2d:ce:b5:02
5c:a6:2d:cf:e7:4e
ac:3a:67:08:29:a2
5c:a6:2d:cf:e7:4d
5c:a6:2d:af:94:4d
5c:a6:2d:af:94:4e
ac:3a:67:08:04:0e
5c:a6:2d:70:7b:41
ac:3a:67:08:04:0d
5c:a6:2d:cf:b3:62
5c:a6:2d:cf:b3:61
5c:a6:2d:70:d0:a2
5c:a6:2d:70:7b:4e
5c:a6:2d:70:7b:4d
5c:a6:2d:af:92:e1
5c:a6:2d:cf:c1:81
5c:a6:2d:cf:c1:82
ac:3a:67:08:28:42
5c:a6:2d:af:92:ee
5c:a6:2d:af:92:ed
5c:a6:2d:45:db:82
5c:a6:2d:70:d0:ae
ac:3a:67:07:f9:21
5c:a6:2d:70:d0:ad
ac:3a:67:08:28:4e
ac:3a:67:08:28:4d
ac:3a:67:08:29:01
ac:3a:67:08:29:02
ac:3a:67:08:04:02
ac:3a:67:08:25:a2
5c:a6:2d:cf:b4:ad
5c:a6:2d:c

5c:a6:2d:cf:e0:0e
5c:a6:2d:cf:e0:0d
5c:a6:2d:cf:c1:62
5c:a6:2d:ce:8e:ad
5c:a6:2d:cf:c1:61
5c:a6:2d:cf:c4:e2
5c:a6:2d:45:dc:a2
5c:a6:2d:ce:c1:e2
5c:a6:2d:ce:c1:e1
ac:3a:67:08:29:01
ac:3a:67:08:25:82
ac:3a:67:08:25:81
ac:3a:67:08:29:02
5c:a6:2d:45:dc:a1
5c:a6:2d:ef:4a:ae
5c:a6:2d:ef:4a:ad
5c:a6:2d:45:dc:ad
5c:a6:2d:45:dc:ae
3c:71:bf:84:d7:51
5c:a6:2d:cf:c4:ed
5c:a6:2d:5d:2c:02
5c:a6:2d:5d:2c:01
5c:a6:2d:45:dc:02
5c:a6:2d:45:dc:01
5c:a6:2d:5d:2c:0e
5c:a6:2d:5d:2c:0d
5c:a6:2d:45:dc:0d
5c:a6:2d:45:dc:0e
5c:a6:2d:86:69:21
5c:a6:2d:86:69:22
ac:3a:67:08:0c:82
5c:a6:2d:cf:e5:a2
5c:a6:2d:cf:bf:c2
ac:3a:67:08:0c:8d
5c:a6:2d:86:69:2e
5c:a6:2d:86:69:2d
5c:a6:2d:cf:e5:ae
5c:a6:2d:cf:e5:ad
ac:3a:67:08:0c:8e
5c:a6:2d:af:91:4e
5c:a6:2d:af:91:4d
5c:a6:2d:5d:2e:cd
5c:a6:2d:5d:2e:ce
5c:a6:2d:cf:e0:02
5c:a6:2d:ce:8e:a2
5c:a6:2d:cf:bf:cd
5c:a6:2d:5d:2e:c1
5c:a6:2d:5d:2e:c2
5c:a6:2d:cf:bf:ce
5c:a6:2d:af:91:42
5c:a6:2d:1e:08:e2
5c:a6:2d:1e:08:e1
5c:a6:2d:cf:b3:e1
5c:a6:2d:cf:b3:e2
5c:a6:2d:cf:b3:ed
5c:a6:2d:c

5c:a6:2d:88:6b:6e
5c:a6:2d:ef:45:2e
5c:a6:2d:ef:45:2d
5c:a6:2d:5d:2b:41
5c:a6:2d:cf:c3:2e
5c:a6:2d:5d:ab:6e
5c:a6:2d:5d:ab:6d
5c:a6:2d:cf:c3:2d
5c:a6:2d:af:8e:0d
5c:a6:2d:45:de:82
ac:3a:67:08:01:8d
ac:3a:67:08:01:8e
76:1f:45:3a:66:fd
5c:a6:2d:86:6d:0e
5c:a6:2d:86:6b:2e
5c:a6:2d:86:6d:0d
5c:a6:2d:85:81:c2
5c:a6:2d:86:6d:a1
5c:a6:2d:86:6d:a2
5c:a6:2d:5d:2d:02
5c:a6:2d:5d:2d:01
5c:a6:2d:86:6b:4e
5c:a6:2d:86:6b:4d
5c:a6:2d:cf:c5:e2
ac:3a:67:07:fc:42
ac:3a:67:07:fc:41
5c:a6:2d:85:81:ce
5c:a6:2d:86:6d:ae
5c:a6:2d:86:6d:ad
5c:a6:2d:cf:e3:42
5c:a6:2d:cf:e3:41
5c:a6:2d:85:81:cd
5c:a6:2d:5d:2d:0e
5c:a6:2d:5d:2d:0d
ac:3a:67:07:fc:4d
ac:3a:67:07:fc:4e
5c:a6:2d:86:6c:22
5c:a6:2d:86:6d:02
5c:a6:2d:cf:c5:ee
5c:a6:2d:cf:c5:ed
5c:a6:2d:cf:e3:4e
ac:3a:67:07:fb:02
5c:a6:2d:cf:e3:4d
5c:a6:2d:eb:9d:81
ac:3a:67:07:fb:0e
ac:3a:67:07:fb:0d
5c:a6:2d:86:6c:2e
5c:a6:2d:af:8e:01
5c:a6:2d:af:8e:02
5c:a6:2d:86:6c:2d
5c:a6:2d:86:6b:22
ac:3a:67:08:01:82
ac:3a:67:08:04:2e
5c:a6:2d:ef:45:2e
ac:3a:67:08:04:2d
5c:a6:2d:c

5c:a6:2d:cf:c1:8e
5c:a6:2d:cf:c1:8d
5c:a6:2d:ce:ab:cd
5c:a6:2d:ce:ab:cc
5c:a6:2d:ce:ab:ce
5c:a6:2d:ef:47:43
5c:a6:2d:ef:47:42
5c:a6:2d:ce:ab:c3
5c:a6:2d:ce:ab:c2
5c:a6:2d:ce:ab:c1
5c:a6:2d:cf:c4:c3
5c:a6:2d:cf:c4:c2
5c:a6:2d:cf:c4:cc
5c:a6:2d:cf:c4:cd
5c:a6:2d:cf:c4:ce
5c:a6:2d:ce:b2:82
5c:a6:2d:ce:b2:83
5c:a6:2d:5d:2b:42
5c:a6:2d:a0:4b:e2
5c:a6:2d:5d:2b:43
5c:a6:2d:a0:4b:ee
5c:a6:2d:a0:4b:ec
5c:a6:2d:a0:4b:ed
5c:a6:2d:ce:c9:6d
ac:3a:67:08:05:6d
ac:3a:67:08:05:6c
ac:3a:67:08:01:81
5c:a6:2d:5d:2b:4d
5c:a6:2d:5d:2b:4e
5c:a6:2d:5d:2b:4c
5c:a6:2d:cf:c1:82
ac:3a:67:08:05:63
5c:a6:2d:cf:c1:83
5c:a6:2d:cf:c1:8c
5c:a6:2d:cf:c1:8d
5c:a6:2d:ce:ab:cd
5c:a6:2d:ce:ab:ce
5c:a6:2d:ef:47:43
5c:a6:2d:ef:47:42
5c:a6:2d:ce:ab:c3
5c:a6:2d:ce:ab:c2
5c:a6:2d:cf:c4:c1
5c:a6:2d:cf:c4:c2
5c:a6:2d:cf:c4:c3
5c:a6:2d:cf:c4:cd
5c:a6:2d:cf:c4:cc
5c:a6:2d:cf:c4:ce
5c:a6:2d:ce:b2:82
5c:a6:2d:ce:b2:83
5c:a6:2d:a0:4b:e3
5c:a6:2d:ce:ab:cc
5c:a6:2d:5d:2b:41
5c:a6:2d:a0:4b:ee
5c:a6:2d:a0:4b:ec
5c:a6:2d:a0:4b:ed
5c:a6:2d:c

5c:a6:2d:ce:ac:8d
5c:a6:2d:ce:ac:82
5c:a6:2d:ce:ac:83
5c:a6:2d:86:69:e2
5c:a6:2d:86:69:e3
5c:a6:2d:86:6a:2c
5c:a6:2d:86:6a:2e
5c:a6:2d:86:6a:2d
5c:a6:2d:86:6a:22
5c:a6:2d:86:6a:23
5c:a6:2d:85:81:e3
5c:a6:2d:85:81:e2
5c:a6:2d:86:6a:43
5c:a6:2d:af:90:62
5c:a6:2d:86:6a:42
5c:a6:2d:af:90:63
5c:a6:2d:86:6c:c3
5c:a6:2d:af:8f:63
5c:a6:2d:86:6c:c2
5c:a6:2d:85:81:ee
5c:a6:2d:85:81:ed
5c:a6:2d:85:81:ec
5c:a6:2d:af:8f:62
5c:a6:2d:86:6c:cd
5c:a6:2d:86:6c:ce
5c:a6:2d:af:90:6c
5c:a6:2d:86:6c:cc
5c:a6:2d:cf:c3:2d
5c:a6:2d:cf:c3:23
5c:a6:2d:86:6a:4c
5c:a6:2d:b5:ca:e2
5c:a6:2d:af:90:6d
5c:a6:2d:af:90:6e
5c:a6:2d:cf:c3:2e
5c:a6:2d:cf:c3:2c
5c:a6:2d:86:6a:4e
5c:a6:2d:b5:ca:e3
5c:a6:2d:eb:9d:83
5c:a6:2d:cf:c3:22
5c:a6:2d:86:6a:4d
5c:a6:2d:eb:9d:82
5c:a6:2d:85:81:cc
5c:a6:2d:85:81:cd
5c:a6:2d:cf:c4:42
5c:a6:2d:af:91:e3
5c:a6:2d:af:91:e2
5c:a6:2d:af:8f:6d
5c:a6:2d:af:8f:6c
5c:a6:2d:cf:c4:43
5c:a6:2d:cf:c4:41
5c:a6:2d:eb:9d:8d
5c:a6:2d:eb:9d:8c
5c:a6:2d:cf:b0:0e
5c:a6:2d:86:6d:23
5c:a6:2d:cf:b0:0d
5c:a6:2d:a

5c:a6:2d:1e:0a:0c
5c:a6:2d:86:6b:ec
5c:a6:2d:86:6b:ed
5c:a6:2d:eb:db:2c
5c:a6:2d:ce:b5:0d
5c:a6:2d:cf:c4:4d
ac:3a:67:08:2a:02
ac:3a:67:08:2a:03
ac:3a:67:08:29:2c
ac:3a:67:08:29:2d
ac:3a:67:08:29:2e
ac:3a:67:08:2a:0d
5c:a6:2d:af:8f:43
5c:a6:2d:45:db:a2
ac:3a:67:08:2a:0e
5c:a6:2d:af:8f:42
ac:3a:67:08:29:22
ac:3a:67:08:2a:0c
ac:3a:67:08:29:23
5c:a6:2d:45:db:ae
5c:a6:2d:86:6c:42
5c:a6:2d:86:6c:43
5c:a6:2d:45:db:ad
5c:a6:2d:45:db:ac
5c:a6:2d:cf:e6:21
5c:a6:2d:cf:e6:22
5c:a6:2d:86:6c:4d
5c:a6:2d:cf:e6:23
5c:a6:2d:86:6c:4c
5c:a6:2d:86:6c:4e
5c:a6:2d:cf:bf:c2
5c:a6:2d:cf:e6:2c
5c:a6:2d:ce:71:02
5c:a6:2d:cf:e6:2e
5c:a6:2d:ce:71:03
5c:a6:2d:cf:df:6c
5c:a6:2d:cf:df:6d
5c:a6:2d:ce:71:0d
5c:a6:2d:cf:bf:ce
5c:a6:2d:86:69:c3
5c:a6:2d:cf:bf:cd
5c:a6:2d:cf:bf:cc
5c:a6:2d:ef:4c:03
5c:a6:2d:ef:4c:02
5c:a6:2d:ce:bf:02
5c:a6:2d:ce:bf:03
5c:a6:2d:5d:ab:62
ac:3a:67:08:26:03
5c:a6:2d:cf:c4:e2
ac:3a:67:07:f9:23
ac:3a:67:07:f9:21
5c:a6:2d:ef:4c:0d
5c:a6:2d:ce:a1:cc
5c:a6:2d:af:8d:8c
5c:a6:2d:ce:bf:0c
ac:3a:67:0